# SVD weight directions are highly interpretable


In [ ]:
# mini nb  #mm

In [13]:
# get everything set up
# more rapidly install node

#!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
# install other dependencies

#!pip install transformers

%pip install  --dry-run  datasets

# install repo with the data
#!git clone https://github.com/BerenMillidge/svd_directions


#aaa
%pip install   --dry-run  tabulate
#%pip install     tabulate
#!bash setup.sh





In [1]:

#%cd svd_directions

import torch
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from copy import deepcopy
from tqdm.auto import tqdm, trange
import re
from collections import defaultdict
from transformers import AutoModelForCausalLM, AutoTokenizer
# utils
import json
from torch import nn
import torch.nn.functional as F
from datasets import load_dataset
from copy import deepcopy
from torch.nn import functional as F
from tabulate import tabulate
from tqdm import tqdm, trange
import functools
import math

# this resets up the site so you don't have to restart the runtime to use pysvelte
#import site
#site.main()
#import pysvelte


sns.set_palette('colorblind')
cmap = sns.color_palette('colorblind')

C:\Users\marti\OneDrive\AI\OLDBIG\svd_directions
